# Revised case normalization for Winterthur 2019

Notes:

- 90 cases can be matched with the DB (if ignoring patientID), otherwise 57.

In [1]:
import sys
sys.path.insert(0, '/home/jovyan/work')

from src.revised_case_normalization.notebook_functions.global_configs import *
from src.revised_case_normalization.notebook_functions.normalize import normalize
from src.revised_case_normalization.notebook_functions.revise import revise
from src.revised_case_normalization.notebook_functions.group import group
from src.revised_case_normalization.notebook_functions.update_db import update_db

In [2]:
file_info = FileInfo(
         os.path.join(ROOT_DIR, 'raw_data/Winterthur.xlsx'),
         'Kantonsspital Winterthur',
         '2019',
         'Änderungen Winterthur 2019')

print(file_info)



FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/Winterthur.xlsx', hospital_name_db='Kantonsspital Winterthur', year='2019', sheet='Änderungen Winterthur 2019')


In [3]:

columns_to_rename = dict(COLUMNS_TO_RENAME)
columns_to_rename.pop("admno")
columns_to_rename['fid'] = CASE_ID_COL

revised_cases_df = normalize(file_info, columns_mapper = columns_to_rename)
revised_cases_df
revised_cases_df.dtypes

2022-11-21 10:01:40.145 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:38 - Read 90 cases for Kantonsspital Winterthur 2019
2022-11-21 10:01:40.150 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:73 - TYPES:
tranche                                                 string
datum                                                   string
case_id                                                 object
patient_id                                              object
kkik                                                    string
gender                                                  string
age_years                                                int64
bfs_code                                                string
duration_of_stay                                         int64
pflegetage neu                                          string
pccl                                                     int64
pccl neu                    

case_id              object
patient_id           object
gender               object
age_years             int64
duration_of_stay      int64
case_id_norm         object
old_pd               object
primary_diagnosis    object
bfs_code             object
pccl                  int64
drg                  object
added_icds           object
removed_icds         object
added_chops          object
removed_chops        object
dtype: object

# Match revised DtoD data with the BfS data from the database

In [4]:
cols_to_join = list(VALIDATION_COLS)
cols_to_join.remove(CASE_ID_COL)
cols_to_join.append(NORM_CASE_ID_COL)
# Ignore the patient ID in this dataset, because it is encrypted
cols_to_join.remove(PATIENT_ID_COL)

In [5]:
cols_to_join = list(VALIDATION_COLS)
cols_to_join.remove(CASE_ID_COL)
cols_to_join.append(NORM_CASE_ID_COL)

# Ignore the patient ID in this dataset, because it is encrypted
cols_to_join.remove(PATIENT_ID_COL)

revised_cases, unmatched = revise(file_info, revised_cases_df, validation_cols=cols_to_join)

2022-11-21 10:01:52.190 | INFO     | src.service.bfs_cases_db_service:get_sociodemographics_for_hospital_year:95 - Read 27873 rows from the DB, for the hospital 'Kantonsspital Winterthur' in 2019


In [6]:
if unmatched.shape[0] > 0:
    display(unmatched)

In [7]:
revisions_update, diagnoses_update, procedures_update = group(revised_cases)
revisions_update[REVISION_DATE_COL] = '2022-12-31'

2022-11-21 10:01:53.776 | INFO     | src.revised_case_normalization.notebook_functions.group:group:86 - Grouping 90 cases ...
2022-11-21 10:01:54.966 | SUCCESS  | src.revised_case_normalization.notebook_functions.group:group:91 - Grouped 90 cases into: 90 revisions, 824 diagnoses rows, 414 procedure rows


In [8]:
revisions_update

,aimedic_id,drg,drg_cost_weight,effective_cost_weight,pccl,revision_date
0,825534,G48C,0.97,0.97,3,2022-12-31
1,825923,H41C,1.27,1.27,3,2022-12-31
2,826912,E65B,0.80,0.80,2,2022-12-31
3,827003,M09A,1.50,1.50,4,2022-12-31
4,827144,E65A,1.18,1.18,4,2022-12-31
...,...,...,...,...,...,...
85,852542,F75B,1.66,1.66,4,2022-12-31
86,852693,I09A,3.71,3.71,4,2022-12-31
87,852800,T60D,1.42,1.42,4,2022-12-31
88,853089,N01C,2.32,2.32,2,2022-12-31


In [9]:
update_db(revisions_update, diagnoses_update, procedures_update)

2022-11-21 10:01:54.989 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:259 - Trying to insert 90 cases into the 'Revisions' table ...
2022-11-21 10:02:01.775 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:296 - Inserted 90 cases into the 'Revisions' table
2022-11-21 10:02:01.782 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:309 - Trying to insert 824 rows into the 'Diagnoses' table ...
2022-11-21 10:02:09.807 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:347 - Inserted 824 rows into the 'Diagnoses' table
2022-11-21 10:02:09.811 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_procedures:358 - Trying to insert 414 rows into the 'Procedures' table ...
2022-11-21 10:02:14.267 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_procedures:404 - Inserted 414 rows into the 'Procedures' table
